In [2]:
from pyspark.sql import SparkSession
from graphframes import GraphFrame

spark = SparkSession.builder.getOrCreate()

In [3]:
vertexesPath = "Spark/Exercises/Ex_57/vertexes.csv"
edgesPath = "Spark/Exercises/Ex_57/edges.csv"
outputPath = "Spark/Exercises/Ex_57/res"

In [4]:
vertexesDF = spark.read.load(vertexesPath, format="csv", header=True, inferSchema=True)
edgesDF = spark.read.load(edgesPath, format="csv", header=True, inferSchema=True)

In [5]:
g = GraphFrame(vertexesDF, edgesDF)

In [8]:
shortestPathsDF = g.shortestPaths(["u1"])
shortestPathsDF.show()

+---+-----+---+---------+
| id| name|age|distances|
+---+-----+---+---------+
| u6| Adel| 36|[u1 -> 3]|
| u3| John| 30|[u1 -> 2]|
| u2|  Bob| 36|[u1 -> 1]|
| u4|David| 29|[u1 -> 1]|
| u5| Paul| 32|[u1 -> 1]|
| u1|Alice| 34|[u1 -> 0]|
| u7| Eddy| 60|       []|
+---+-----+---+---------+



In [9]:
shortestPathsDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- distances: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = false)



In [15]:
acceptableShortestPathsU1DF = shortestPathsDF.filter("id <> 'u1' and distances['u1'] <= 2")
acceptableShortestPathsU1DF.show()

+---+-----+---+---------+
| id| name|age|distances|
+---+-----+---+---------+
| u3| John| 30|[u1 -> 2]|
| u2|  Bob| 36|[u1 -> 1]|
| u4|David| 29|[u1 -> 1]|
| u5| Paul| 32|[u1 -> 1]|
+---+-----+---+---------+



In [16]:
finalDF = acceptableShortestPathsU1DF.selectExpr("name", "distances['u1'] as numHops")
finalDF.show()

+-----+-------+
| name|numHops|
+-----+-------+
| John|      2|
|  Bob|      1|
|David|      1|
| Paul|      1|
+-----+-------+



In [17]:
finalDF.write.csv(outputPath,header=True)